### RNN, LSTM and S2S

#### 前馈神经网络 vs RNN
1 DAG vs 有环的计算图
2 信号往同一个方向传 vs 信号被传回到同一个神经元
3 每一层有独立的变量 vs 每一步共用变量

#### BPTT
1 每一步的梯度求和后更新变量
2 步数很多时将需要很多的计算

#### Truncated BPTT
使用有限的步数，提高训练速度

因为梯度消失/梯度爆炸，RNN 并不能保存长期的记录

#### LSTM
- input gate: 这次的输入需要放进来多少
- forget gate：上次的状态需要考虑多少
- output gate：隐藏的状态需要放出多少
- candidate gate
- memory cell

#### RNN 能做什么
- 计算句子的相似度
- 机器翻译
- 文本生成
- 文本归纳
- 看图说话 - 将 CNN 的特征作为输入

#### 语言模型
- word level
- char level

#### 语言模型缺点
- n-gram
  - 基于先前的 n-gram 来推测下一个词是什么
  - 大量的词典
  - 没办法生成词典外的词
  - 需要大量内存
- char level
  - 非常小的词典
  - 不需要 word embedding
  - 训练快速
  - 但效率低，容易出现瞎说



In [ ]:
# 基本的 tf.nn.rnn_cell
# BasicRNNCell 基本的 RNN 单元
# RNNCell: Abstract object representing an RNN cell
# BasicLSTMCell: 基本的 LSTM 单元
# LSTMCell: LSTM 单元
# GRUCell: GRU 单元

# 单个单元
cell = tf.nn.rnn_cell.GRUCell(hidden_size)
# 将单元组合
layers = [tf.nn.rnn_cell.GRUCell(size) for size in hidden_sizes] # 每个隐藏层的状态 size
cells = tf.nn.rnn_cell.MultiRNNCell(layers) # 多层的 RNN
# dynamic_rnn，使用 tf.While 来动态构建图。并且能喂给变长的 batch
output, out_state = tf.nn.dynamic_rnn(cell, seq, length, initial_state)
# 对 seq 做 padding，loss 只计算非 padding 的
full_loss = tf.nn.softmax_cross_entropy_with_logits(preds, labels)
loss = tf.reduce_mean(tf.boolean_mask(full_loss, mask))
# 告诉模型真实的 seq 长度，让模型只预测真实的 token
cell = tf.nn.rnn_cell.GRUCell(hidden_size)
rnn_cells = tf.nn.rnn_cell.MultiRNNCell([cell] * num_layers)
tf.reduce_sum(tf.reduce_max(tf.sign(seq), 2), 1)
output, out_state = tf.nn.dynamic_rnn(cell, seq, length, initial_state)


In [5]:
# tricks
# 使用不同的激活函数
tf.nn.relu
tf.nn.relu6
tf.nn.crelu
tf.nn.elu
# 更多的激活函数
tf.nn.softplus
tf.nn.softsign
tf.nn.bias_add
tf.sigmoid
tf.tanh
# 梯度截断
gradients = tf.gradients(cost, tf.trainable_variables())
clipped_gradients, _ = tf.clip_by_global_norm(gradients, max_grad_norm) 
optimizer = tf.train.AdamOptimizer(learning_rate)
train_op = optimizer.apply_gradients(zip(gradients, trainables))
# 学习率的退火
learning_rate = tf.train.exponential_decay(init_lr, global_step, decay_steps, decay_rate, staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate)
# dropout 防止过拟合
hidden_layer = tf.nn.dropout(hidden_layer, keep_prob)
cell = tf.nn.rnn_cell.GRUCell(hidden_size)
cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=keep_prob)
